In [2]:
import os
import pandas as pd
from torchvision.io import read_image
import torch
import numpy as np
from PIL import Image
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.enable_eager_execution()
tf.enable_eager_execution()
import matplotlib.pyplot as plt
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
import cv2
from scipy import ndimage


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
pip install opencv-python

In [ ]:


img_resize = [28,28]
epoch_num = None   # dataset.repeat() 的参数，设置为None，可以不断取数
# 传入图片名，返回正则化后的图片的像素值
def read_img(img_name, lable):
    image = tf.read_file(img_name)
    image = tf.image.decode_jpeg(image,channels=3)
    image = cv2.cvtColor(image.numpy(), cv2.COLOR_BGR2GRAY)
    image=image/255

    image=torch.from_numpy(image)
    image= tf.cast(image, tf.float32)
    return image,lable
    return image,lable
# 传入图片所在的文件夹，图片名含有图片的lable，返回利用文件夹中图片创建的dataset
def create_dataset(path):
    files = os.listdir(path)  # 列出文件夹中所有的图片
    img_names = []
    lables = []
    for f in files:
        img_names.append(os.path.join(path,f))   # 图片的完整路径append到文件名list中
        order = int(f.split('.')[0])

        lables.append(d[1][order])  # 根据规则得到图片的lable

    img_names = tf.convert_to_tensor(img_names, dtype=tf.string)
    lables = tf.convert_to_tensor(lables, dtype=tf.int64)  # 将图片名list和lable的list转换成Tensor类型
    dataset = tf.data.Dataset.from_tensor_slices((img_names,lables))  # 创建dataset，传入的需要是tensor类型
    dataset = dataset.map(read_img)   # 传入read_img函数，将图片名转为像素　　 　　# 将dataset打乱，设置一次获取batch_size条数据 

    return dataset


In [ ]:

d = pd.read_csv('E:/21Cam/data/lungcancer_image/valid.csv',encoding='utf-8',header=None,index_col=0)
gray_valid_set_raw = create_dataset('E:/21Cam/data/lungcancer_image/ajust_image/gray/valid')   # 图片所在的路径为./img
iterator = gray_valid_set_raw.make_one_shot_iterator()
one_element = iterator.get_next()  # 创建dataset是batch_size 为多少这里一次就能获取多少个数据

In [ ]:
gray_valid_set_raw
color_test_set_raw 
gray_train_set_raw

In [ ]:
n_classes=3
dataset_size=555
class_names=['POORLY DIFFERENTIATED','Moderate Differentiation','WELL DIFFERENTIATED']

In [ ]:

batch_size = 22
gray_train_set = gray_train_set_raw.shuffle(500).repeat().batch(batch_size).prefetch(1)
gray_valid_set = gray_valid_set_raw.batch(batch_size).prefetch(1)
gray_test_set = gray_test_set_raw.batch(batch_size).prefetch(1)

In [ ]:
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10, 10))
index = 0

for image, label in gray_valid_set_raw.take(4):

    index += 1

    plt.subplot(2, 2, index)
    plt.imshow(image[0])
    plt.title("Class: {}".format(class_names[label]))
    plt.axis("off")

plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
for X_batch, y_batch in gray_train_set.take(1):
    for index in range(9):
        plt.subplot(3, 3, index + 1)
    
        plt.imshow(X_batch[index])
        plt.title("Class: {}".format(class_names[y_batch[index]]))
        plt.axis("off")

plt.show()

In [ ]:
base_model1 = keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg1 = keras.layers.GlobalAveragePooling2D()(base_model1.output)
output1 = keras.layers.Dense(n_classes, activation="softmax")(avg1)
model1 = keras.models.Model(inputs=base_model1.input, outputs=output1)

In [ ]:
for layer in base_model1.layers:
    layer.trainable = True

optimizer1 = keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9,nesterov=True, decay=1e-4)
model1.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer1,metrics=["accuracy"])
history1 = model1.fit(gray_train_set,steps_per_epoch=int(391 / batch_size),
                    validation_data=gray_valid_set,
                    validation_steps=int(107 / batch_size),
                    epochs=30)

In [ ]:

score = model1.evaluate(gray_test_set)
